In [ ]:
import os
import pandas as pd

In [ ]:
!pip install ultralytics

In [ ]:
import yaml
from ultralytics import YOLO

# Step 1: Update data.yaml file
original_yaml_path = "/kaggle/input/can-body-lid/Can-Body-Lid/data.yaml"
updated_yaml_path = "/kaggle/working/data.yaml"

# Load and modify the YAML file
with open(original_yaml_path, "r") as file:
    data = yaml.safe_load(file)

# Update paths in the YAML file
data['train'] = "/kaggle/input/can-body-lid/Can-Body-Lid/train/images"
data['val'] = "/kaggle/input/can-body-lid/Can-Body-Lid/valid/images"
data['test'] = "/kaggle/input/can-body-lid/Can-Body-Lid/test/images"

# Save the updated YAML to /kaggle/working
with open(updated_yaml_path, "w") as file:
    yaml.safe_dump(data, file)

print(f"Updated data.yaml saved to: {updated_yaml_path}")

# Step 2: Train YOLOv8 Model
model = YOLO('yolov8n.pt')

# Train the model with 200 epochs and image size 299x299
model.train(data=updated_yaml_path, epochs=30, imgsz=299, project='/kaggle/working', name='yolov8_can_body_lid')

# Step 3: Validate the Model
metrics = model.val()

# Extract Metrics from the Results
print("\nValidation Metrics:")
results = metrics.results_dict
print(f"mAP@0.5: {results['mAP50']:.4f}")
print(f"mAP@0.5:0.95: {results['mAP50-95']:.4f}")
print(f"Precision: {results['precision']:.4f}")
print(f"Recall: {results['recall']:.4f}")

# Step 4: Predict on the Test Dataset
test_images_path = "/kaggle/input/can-body-lid/Can-Body-Lid/test/images"
test_results = model.predict(source=test_images_path, save=True, save_txt=True, project='/kaggle/working', name='yolov8_test_predictions')

print("\nTest Results:")
print(f"Number of Predictions: {len(test_results)}")
print("Bounding boxes saved with images in '/kaggle/working/yolov8_test_predictions/'.")

# Step 5: Export the Model to ONNX and TorchScript
export_formats = ['onnx', 'torchscript']  # Export formats
for format in export_formats:
    export_path = model.export(format=format, project='/kaggle/working', name=f'yolov8_can_body_lid_{format}')
    print(f"Model exported to {format.upper()} format at: {export_path}")


In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt
from ultralytics import YOLO

# Step 1: Define Test Images Path
test_images_path = "/kaggle/input/can-body-lid/Can-Body-Lid/test/images"

# Step 2: Predict on the Test Dataset
model = YOLO('yolov8n.pt')  # Load the trained model (ensure it's trained before this step)
test_results = model.predict(
    source=test_images_path,
    save=True,
    save_txt=True,
    project='/kaggle/working',
    name='yolov8_test_predictions'
)

# Step 3: Path to the Folder with Processed Images
processed_images_path = "/kaggle/working/yolov8_test_predictions/predict/"

# Step 4: Display All Images with Bounding Boxes
image_files = [f for f in os.listdir(processed_images_path) if f.endswith('.jpg')]

print("Displaying test images with bounding boxes:")

for image_file in image_files:
    # Load the image
    image_path = os.path.join(processed_images_path, image_file)
    img = Image.open(image_path)

    # Display the image with bounding boxes
    plt.figure(figsize=(8, 8))
    plt.imshow(img)
    plt.axis('off')
    plt.title(image_file)
    plt.show()
